In [2]:
import csv
import random
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import concurrent
from tqdm import tqdm

In [3]:
model_name = 'llmrails/ember-v1'
model = SentenceTransformer(model_name)

/Users/undinagisladottir/opt/anaconda3/envs/llm_cpus/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


## Overlapping Dataset

In [4]:
overlap_data = pd.read_csv('data/overlap_data.csv').drop_duplicates()
overlap_data.head()

,drug,adverse_event,comp_term,is_overlapping
0,XEOMIN,Hypersensitivity,Precautions Dysphagia and,0
1,XEOMIN,Hypersensitivity,reactions to XEOMIN are discussed in greater d...,0
2,XEOMIN,Dysphagia,Dysphagia and Breathing Difficulties in Treatment,1
3,XEOMIN,Dysphagia,Dysphagia and,1
5,XEOMIN,Dysphagia,Dysphagia and Breathing,1


In [25]:
def run_iteration(row):
    try:
        ade_embed = model.encode(row.adverse_event)
        comp_term = model.encode(row.comp_term)
        similarity = cos_sim(ade_embed, comp_term)[0]
        return [row.drug, row.adverse_event, row.comp_term, row.is_overlapping, float(similarity)]
    except:
        return None

NameError: name 'overlap_data' is not defined

In [37]:
run_iteration(overlap_data.iloc[0])

['XEOMIN',
 'Hypersensitivity',
 'Precautions Dysphagia and',
 0,
 0.5108517408370972]

In [49]:
with concurrent.futures.ThreadPoolExecutor() as exec:
    results = list(tqdm(
		exec.map(run_iteration, overlap_data.itertuples()), 
		total=len(overlap_data)
	))

 48%|████▊     | 22419/46840 [16:54:00<18:24:33,  2.71s/it]     


KeyboardInterrupt: 

: 

In [47]:
res_sim = pd.DataFrame(
    results,
    columns=['drug_name', 'adverse_event', 'comp_term', 'is_overlapping', 'similarity']
)

In [48]:
res_sim

,drug_name,adverse_event,comp_term,is_overlapping,similarity
0,XEOMIN,Hypersensitivity,Precautions Dysphagia and,0,0.510852
1,XEOMIN,Hypersensitivity,reactions to XEOMIN are discussed in greater d...,0,0.554153
2,XEOMIN,Dysphagia,Dysphagia and Breathing Difficulties in Treatment,1,0.766292
3,XEOMIN,Dysphagia,Dysphagia and,1,0.864872
4,XEOMIN,Dysphagia,Dysphagia and Breathing,1,0.821762
...,...,...,...,...,...
95,XEOMIN,dysphagia,Units or Units dysphagia neck pain muscle weak...,1,0.672436
96,XEOMIN,neck pain,included neck,1,0.661478
97,XEOMIN,neck pain,Units,0,0.441835
98,XEOMIN,neck pain,Units or Units included neck,1,0.609689


In [17]:
temp = overlap_data[:(ind)]
temp['similarity'] = [float(x[0]) for x in similarities]
temp.head()

/var/folders/sv/s1jkxt011xn9v02w3w30vx640000gn/T/ipykernel_33216/904460465.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['similarity'] = [float(x[0]) for x in similarities]


,drug,adverse_event,comp_term,is_overlapping,similarity
0,XEOMIN,Hypersensitivity,Precautions Dysphagia and,0,0.510852
1,XEOMIN,Hypersensitivity,reactions to XEOMIN are discussed in greater d...,0,0.554153
2,XEOMIN,Dysphagia,Dysphagia and Breathing Difficulties in Treatment,1,0.766292
3,XEOMIN,Dysphagia,Dysphagia and,1,0.864872
5,XEOMIN,Dysphagia,Dysphagia and Breathing,1,0.821762


In [53]:
overlap_data['ade_embed'] = list(model.encode(overlap_data['adverse_event'].tolist()))
overlap_data['comp_embed'] = list(model.encode(overlap_data['comp_term'].tolist()))
overlap_data.head()

KeyboardInterrupt: 

## GPT output compared to manual

In [39]:
gpt_output = pd.read_csv('results/OpenAI_gpt-3.5-turbo-0125_fatal-prompt-v2_pharmexpert-v0_train_run0.csv',
                         usecols = ['drug_name', 'section_name', 'gpt_output'])
gpt_output['gpt_output'] = gpt_output['gpt_output'].str.replace('.', '').str.split(', ')
gpt_output = gpt_output.explode('gpt_output').reset_index(drop = True).drop_duplicates()
manual_file = pd.read_csv('data/train_drug_label_text_manual_ades.csv',
                          usecols = ['drug_name', 'reaction_string', 'meddra_pt', 'section_name',
                                     'discontinuous_term', 'negated_term', 'meddra_exact_term']).drop_duplicates()
gpt_output.head()

,drug_name,section_name,gpt_output
0,KYPROLIS,adverse reactions,Cardiac Toxicities
1,KYPROLIS,adverse reactions,Acute Renal Failure
2,KYPROLIS,adverse reactions,Tumor Lysis Syndrome
3,KYPROLIS,adverse reactions,Pulmonary Toxicity
4,KYPROLIS,adverse reactions,Pulmonary Hypertension


In [41]:
gpt_embeds = model.encode(gpt_output['gpt_output'].tolist())
gpt_output['embeds'] = list(gpt_embeds)

In [42]:
man_embeds = model.encode(manual_file['reaction_string'].tolist())
manual_file['embeds'] = list(man_embeds)

In [30]:
for drug in gpt_output.drug_name.unique():
    gpt_sub = gpt_output.loc[gpt_output.drug_name == drug]

,drug_name,section_name,gpt_output,embeds
0,KYPROLIS,adverse reactions,Cardiac Toxicities,"[-0.11013074, 0.6761941, 0.7869268, 0.3508221,..."
1,KYPROLIS,adverse reactions,Acute Renal Failure,"[0.010086697, 0.9340907, -0.20226005, -0.00631..."
2,KYPROLIS,adverse reactions,Tumor Lysis Syndrome,"[0.768426, -0.20337762, 0.062222198, 0.8446492..."
3,KYPROLIS,adverse reactions,Pulmonary Toxicity,"[-0.09318504, 0.15409195, 0.7109283, -0.237119..."
4,KYPROLIS,adverse reactions,Pulmonary Hypertension,"[-0.39612737, 0.2627319, -0.0540106, -0.066323..."
